In [1]:
import numpy as np
from keras.models import Sequential, model_from_json
from keras.preprocessing import sequence
from keras.layers import Dense, Embedding, LSTM, Dropout
import time
import pickle
import datetime

/Users/yao-chiehhu/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Preprocessed Data

In [2]:
with open('data.pickle', 'rb') as handle:
    fullHeadTailDataList = pickle.load(handle)

## Prepare two datasets: Enrol+Quota+Wait and Wait-only

In [3]:
fullHeadTailDataListEQW = []
for item in fullHeadTailDataList:
    fullHeadTailDataListEQW.append([item['enrol'],item['quota'],item['wait']])

In [4]:
fullHeadTailDataListWAIT = []
for item in fullHeadTailDataList:
    fullHeadTailDataListWAIT.append(item['wait'])

In [64]:
def getPredictableData(_datetime):
    index = -1
    # Capture datetime
    for ind,row in enumerate(fullHeadTailDataList):
        if (row['datetime'] == _datetime):
            index = ind
            
    print("Index of Predictable Data:", index)
    
    
    # Found corresponding datetime
    if (index != -1):
        res = np.asarray(fullHeadTailDataListEQW[index-1:index+1])
        print("Predictable Data:", res)
        return np.swapaxes(res,0,1).reshape(1,3,2)
    else:
        return np.zeros(1)

## Create lookback 

In [6]:
def create_dataset(dataset, look_back=1): 
    X, Y = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:i+look_back]
        X.append(a)
        Y.append(dataset[i + look_back]) 
    return np.array(X), np.array(Y)

## Save model

In [7]:
def saveModel(model, modelFilenamePrefix):
    structureFilename = modelFilenamePrefix + ".json" 
    model_json = model.to_json()
    with open(structureFilename, "w") as f:
        f.write(model_json)
        
    weightFilename = modelFilenamePrefix + ".h5" 
    model.save_weights(weightFilename)

## Read model

In [8]:
def readModel(modelFilenamePrefix):
    structureFilename = modelFilenamePrefix + ".json" 
    with open(structureFilename, "r") as f:
        model_json = f.read()
    model = model_from_json(model_json)

    weightFilename = modelFilenamePrefix + ".h5" 
    model.load_weights(weightFilename)
    return model

In [9]:
_,dataY = create_dataset(fullHeadTailDataListWAIT, 2)
dataX,_ = create_dataset(fullHeadTailDataListEQW, 2)

In [10]:
print(dataX.shape)
print(dataY.shape)

(990, 2, 3)
(990,)


## Train 1

In [83]:
def train1(X,Y,_dim):
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    print(X[0])

    model = Sequential()
    model.add(Dense(8, input_dim=_dim, activation='relu')) 
    model.add(Dense(1, activation='relu'))

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["mae"])
    model.fit(X, Y, epochs=50, batch_size=4, validation_split=0.2)

    scores = model.evaluate(X, Y)
    print("{}: {}".format(model.metrics_names[1], scores[1]*100))

    return model

In [84]:
model1 = train1(dataX.reshape(len(dataX),6), dataY, 6)

[ 48. 125.   0.  48. 125.   0.]
Train on 792 samples, validate on 198 samples
Epoch 1/50
792/792 [==============================] - 1s 1ms/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 2/50
792/792 [==============================] - 0s 346us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 3/50
792/792 [==============================] - 0s 334us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 4/50
792/792 [==============================] - 0s 365us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 5/50
792/792 [==============================] - 0s 357us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 6/50
792/792 [==============================] - 

792/792 [==============================] - 0s 340us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 48/50
792/792 [==============================] - 0s 352us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 49/50
792/792 [==============================] - 0s 337us/step - loss: -751.2849 - mean_absolute_error: 47.1452 - val_loss: -354.1947 - val_mean_absolute_error: 22.2172
Epoch 50/50
990/990 [==============================] - 0s 18us/step
mean_absolute_error: 4215.95959518895


## Train 4

## Train 5

In [43]:
def trainLSTM(X,Y, timestep=1, dim=1):
    # Reshape to fit the LSTM model
    X = np.swapaxes(X,1,2)
        
    # Model
    model = Sequential()
    model.add(LSTM(4, input_shape=(timestep,dim)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["mae"])
    model.fit(X, Y, epochs=50, batch_size=8, validation_split=0.2)

    # Ealuation
    scores = model.evaluate(X, Y)
    print("{}: {}".format(model.metrics_names[1], scores[1]*100))
    
    return model

In [44]:
model = trainLSTM(dataX,dataY,3,2)

[[[ 48.  48.]
  [125. 125.]
  [  0.   0.]]

 [[ 48.  60.]
  [125. 125.]
  [  0.   0.]]

 [[ 60.  59.]
  [125. 125.]
  [  0.   0.]]

 ...

 [[138. 138.]
  [140. 140.]
  [ 17.  17.]]

 [[138. 138.]
  [140. 140.]
  [ 17.  17.]]

 [[138. 138.]
  [140. 140.]
  [ 17.  17.]]]
Train on 792 samples, validate on 198 samples
Epoch 1/50
792/792 [==============================] - 2s 2ms/step - loss: 45.0454 - mean_absolute_error: 47.8026 - val_loss: 13.0264 - val_mean_absolute_error: 22.8543
Epoch 2/50
792/792 [==============================] - 0s 583us/step - loss: 9.0393 - mean_absolute_error: 47.6555 - val_loss: -2.5552 - val_mean_absolute_error: 22.6874
Epoch 3/50
792/792 [==============================] - 0s 530us/step - loss: -106.2707 - mean_absolute_error: 47.4438 - val_loss: -102.9841 - val_mean_absolute_error: 22.2542
Epoch 4/50
792/792 [==============================] - 0s 544us/step - loss: -570.0164 - mean_absolute_error: 47.0513 - val_loss: -353.1739 - val_mean_absolute_error: 22.1129

792/792 [==============================] - 0s 530us/step - loss: -751.2849 - mean_absolute_error: 45.1930 - val_loss: -354.1947 - val_mean_absolute_error: 20.4446
Epoch 47/50
792/792 [==============================] - 0s 533us/step - loss: -751.3502 - mean_absolute_error: 45.1976 - val_loss: -354.1947 - val_mean_absolute_error: 20.4599
Epoch 48/50
792/792 [==============================] - 0s 493us/step - loss: -750.2236 - mean_absolute_error: 45.2210 - val_loss: -354.1947 - val_mean_absolute_error: 20.4691
Epoch 49/50
792/792 [==============================] - 0s 543us/step - loss: -750.4371 - mean_absolute_error: 45.2468 - val_loss: -354.1947 - val_mean_absolute_error: 20.4705
Epoch 50/50
990/990 [==============================] - 0s 37us/step
mean_absolute_error: 4027.8533161047735


In [45]:
saveModel(model, "test")

In [46]:
test_model = readModel("test")

In [72]:
predictableData = getPredictableData(datetime.datetime(2018, 1, 27, 14, 30))

Index of Predictable Data: 107
Predictable Data: [[135 135  61]
 [135 135  61]]


In [73]:
print(predictableData.shape)

(1, 3, 2)


In [74]:
model.predict(predictableData)

array([[2.9607518]], dtype=float32)

In [68]:
dataX[0]

array([[ 48., 125.,   0.],
       [ 48., 125.,   0.]])

In [38]:
tmp = np.asarray(dataX[0:1])
# tmp = np.reshape(tmp, (len(tmp),3,2))
tmp = np.swapaxes(tmp,1,2)
print(tmp)
print(tmp[0])
print(tmp[0][0])
print(tmp[0][0][0])

[[[ 48.  48.]
  [125. 125.]
  [  0.   0.]]]
[[ 48.  48.]
 [125. 125.]
 [  0.   0.]]
[48. 48.]
48.0


In [24]:
tmp = np.asarray(dataX[0:1])
tmp[0].transpose()
# tmp = np.reshape(tmp, (len(tmp),3,2))
print(tmp)

[[[ 48. 125.   0.]
  [ 48. 125.   0.]]]


In [25]:
model.predict(tmp)

ValueError: Error when checking : expected lstm_1_input to have shape (3, 2) but got array with shape (2, 3)